This notebook adapts the [Captum tutorial for question answering](https://captum.ai/tutorials/Bert_SQUAD_Interpret) and refactors it into the longformer sequence classification task. Specifically, this notebook focuses on using the model's embeddings to get token attributions for the examples of your choice, or the entire dataset if needed. By doing so, we can visualize which tokens have the most influence in the model's prediction, and find out the k tokens with the most influence at helping the model predict correctly as well as incorrectly.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import dependencies

In [ ]:
pip install transformers --quiet

     |████████████████████████████████| 4.7 MB 6.8 MB/s 
     |████████████████████████████████| 101 kB 10.2 MB/s 
     |████████████████████████████████| 6.6 MB 58.0 MB/s 
     |████████████████████████████████| 596 kB 91.7 MB/s 


In [ ]:
pip install captum --quiet

     |████████████████████████████████| 1.4 MB 6.8 MB/s 


In [ ]:
pip install datasets --quiet

     |████████████████████████████████| 365 kB 7.6 MB/s 
     |████████████████████████████████| 115 kB 80.6 MB/s 
     |████████████████████████████████| 141 kB 76.0 MB/s 
     |████████████████████████████████| 212 kB 71.2 MB/s 
     |████████████████████████████████| 127 kB 86.7 MB/s 


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch
import pandas as pd

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

Here we are importing the model and tokenizer and letting the model use our GPU to run. Please change model path, and tokenizer to whichever one you wish to use.

In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model

# load model
test = torch.load("/content/drive/MyDrive/fakeclinicalnotes/models/full_augmented_lr2e-5_dropout3_10_trained_threshold.pt")
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', state_dict=test['state_dict'], num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['longformer_model.encoder.layer.3.output.LayerNorm.bias', 'longformer_model.encoder.layer.8.output.LayerNorm.weight', 'fc.bias', 'longformer_model.encoder.layer.7.attention.output.dense.weight', 'longformer_model.encoder.layer.10.attention.self.value.bias', 'longformer_model.encoder.layer.1.attention.self.value.bias', 'longformer_model.encoder.layer.7.attention.self.key_global.weight', 'longformer_model.encoder.layer.5.attention.self.query.bias', 'longformer_model.encoder.layer.6.attention.self.query.bias', 'longformer_model.encoder.layer.1.attention.self.query_global.bias', 'longformer_model.encoder.layer.9.attention.self.key_global.bias', 'longformer_model.encoder.layer.10.attention.self.query.weight', 'longformer_model.encoder.layer.0.attention.self.query.weight', 'longformer_model.encoder.layer.10.output.dense.bias', 'longformer_model.encoder.la

Create functions that give us the input ids and the position ids for the text we want to examine along with the baselines for integrated gradients.

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [ ]:
max_length = 2046
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)

    #taken from the longformer implementation
    mask = input_ids.ne(ref_token_id).int()
    incremental_indices = torch.cumsum(mask, dim=1).type_as(mask) * mask
    position_ids = incremental_indices.long().squeeze() + ref_token_id

    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    position_ids = position_ids[:, :seq_length]
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

### Import Dataset

Here we import the papers dataset

In [ ]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402datafake")["train"]

Downloading:   0%|          | 0.00/613 [00:00<?, ?B/s]

Using custom data configuration danielhou13--cogs402datafake-f5349e6cf83e41d8


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402datafake-f5349e6cf83e41d8/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Here we import the news dataset

In [ ]:
# cogs402_ds = load_dataset("danielhou13/cogs402dataset2")["validation"]

## Getting the Attributions

A custom forward function that returns the softmaxed logits, which are the class probabilities that the model uses for prediction.

In [ ]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return output.logits

In [ ]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None, attention_mask=None):
    preds = predict(inputs,
                   position_ids=position_ids,
                   attention_mask=attention_mask
                   )
    return torch.softmax(preds, dim = 1)

A helper function to summarize attributions for each word token in the sequence.

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.norm(attributions)
    return attributions

Perform Layer Integrated Gradients using the longformer's embeddings.

In [ ]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

This function will let us get the example and the baseline inputs in order to perform integrated gradients, and add the attributions to our visualization tool. Additionally, we will add the attributions and tokens for each example into an array so we can use them when we want to further examine the attributions scores for each example. More information about the integrated gradients function can be found [here](https://captum.ai/api/layer.html#layer-integrated-gradients).

In [ ]:
vis_data_records = []
all_attributions = {}
all_tokens = {}
all_deltas = {}

In [ ]:
# Takes in dataset and example number
def get_token_attributions(dataset, example):
  text = dataset['text'][example]
  label = dataset['labels'][example]

  # get the inputs, position ids, attention mask, and the baselines
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  attention_mask = construct_attention_mask(input_ids)

  #get the tokens
  indices = input_ids[0].detach().tolist()
  all_tokens_curr = tokenizer.convert_ids_to_tokens(indices)
  all_tokens[str(example)] = all_tokens_curr

  #perform integrated gradients
  attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    additional_forward_args=(position_ids, attention_mask),
                                    target=1,
                                    n_steps=500,
                                    internal_batch_size = 2)

  # We want one value for every token.
  attributions_sum = summarize_attributions(attributions)

  # store the values in our dictionary
  all_attributions[str(example)] = attributions_sum
  all_deltas[str(example)] = attributions_sum

  # get the score for our visualization
  score = predict(input_ids, position_ids, attention_mask)

  # storing couple samples in an array for visualization purposes
  # requires array of attributions, prediction score, predicted class, true class 
  # the label you want your attributions to associate positive with, the attribution score
  # the tokens, and the delta if you have it.
  vis_data_records.append(viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.softmax(score, dim = 1).max(),
                        torch.argmax(torch.softmax(score, dim = 1)),
                        label,
                        str(1),
                        attributions_sum.sum(),       
                        all_tokens_curr,
                        delta)
  )

Here we are taking some examples from the Papers datasets.

In [ ]:
get_token_attributions(cogs402_ds, 7)

This function allows us to display our attributions in a manner that is easy to read. We can see the attributions of the word overlayed on top of their respective token. The green colour represents positive attributions (i.e. the model is attributing this token to influential for predicting the positive class) while the red colour represents negative attributions. 

In [ ]:
# # storing couple samples in an array for visualization purposes
# score_vis = viz.VisualizationDataRecord(
#                         attributions_sum,
#                         torch.softmax(score, dim = 1).max(),
#                         torch.argmax(torch.softmax(score, dim = 1)),
#                         label,
#                         str(1),
#                         attributions_sum.sum(),       
#                         all_tokens,
#                         delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
_ = viz.visualize_text(vis_data_records)

 Visualization For Score 


## Further Examination of the Attributions

Next we might want to look in-depth about the attribution scores for each token of an example. We saved the attributions for the examples we looked at above, so we can easily retrieve the attributions. We also grab the examples because we want to know what tokens the attributions are associated with.

Both lists are of shape: (seq_len)

In [ ]:
example = 7
attributions_sum = all_attributions[f"{example}"]
all_tokens2 = all_tokens[f"{example}"]

These functions return which words had the strongest (most positive and most negative) attributions. Change the number of tokens you wish to visualize for your needs. It takes in the attributions and the tokens we grabbed in the previous cell and returns 3 lists: the topk (or bottomk) attributions, their respective token and their position.

Note: Remember that the attributions are with respect to the positive class, so the most impact tokens that helped the model predict the negative class will be in the botk attributed tokens.

In [ ]:
def get_topk_attributed_tokens(attrs, all_tokens, k=20):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [ ]:
def get_botk_attributed_tokens(attrs, all_tokens, k=20):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

Convert the values, index of the values, and the token into a pandas Dataframe for visualization. It will be sorted by highest value for attributions to lowest. Alternatively, if youre looking for the most negative attributions, it goes from lowest to highest.

In [ ]:
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum.cpu(), all_tokens2)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum.cpu(), all_tokens2)

df_high = pd.DataFrame({'Word': top_words_start, 'index':top_word_ind_start, 'attribution': top_words_val_start})

df_low = pd.DataFrame({'Word': bot_words_start, 'index':bot_word_ind_start, 'attribution': bot_words_val_start})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

Here we display our top k positively and negatively attributed tokens for our example.

In [ ]:
df_high

,Word,index,attribution
0,Ġaut,649,0.073457
1,F,150,0.069968
2,Ġprivileges,530,0.066580
3,Ġautistic,1709,0.066424
4,Ġevidence,1633,0.066358
5,Ġfactors,1802,0.065642
6,ĠAt,1703,0.063147
7,Ġfrom,536,0.060302
8,:,205,0.060179
9,Ġpursuit,1475,0.060011


In [ ]:
df_low

,Word,index,attribution
0,Ġthat,1643,-0.078480
1,Ġthat,601,-0.071664
2,Ġthat,1765,-0.070016
3,Ġthat,1194,-0.069526
4,Ġthat,1016,-0.069123
5,Ġthat,960,-0.067535
6,Ġthat,1876,-0.066175
7,Ġthat,966,-0.065946
8,Ġand,297,-0.064247
9,Ġgetting,539,-0.063121


In [ ]:
d = {"tokens":all_tokens2, "attribution":attributions_sum[:len(all_tokens2)].cpu()}

We notice that there are many repeating tokens in each example that have different positions. While we might want to know how the position plays into the attributions, if we want to know strictly based on the tokens itself, we can add all the duplicate tokens together to get the aggregate attribution for each token. Therefore, we aggregate the attributions strictly based on token type.

In [ ]:
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [ ]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False).reset_index()
highest_attrib_tokens[:10]

,tokens,attribution
0,.,1.070852
1,Ġwas,0.628402
2,Ġthe,0.523456
3,Ġto,0.500308
4,Ġof,0.389774
5,Ġbe,0.348745
6,Ġa,0.338468
7,Ġfrom,0.337042
8,per,0.315743
9,:,0.305437


In [ ]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution']).reset_index()
lowest_attrib_tokens[:10]

,tokens,attribution
0,Ġthat,-2.287376
1,Ġand,-1.698054
2,Ġ,-1.034440
3,ĉ,-0.385827
4,Ġwith,-0.353948
5,Ġbut,-0.312914
6,ĠI,-0.297695
7,Ġby,-0.296279
8,Ġit,-0.281805
9,Ġany,-0.259322


## Masking the stopwords and non-alpha tokens

There may be some stopwords or punctuations in our top attributed tokens, so now that we have the list of the highest and lowest, we can identify interesting keywords.

In [ ]:
import nltk
from transformers import AutoTokenizer
nltk.download('stopwords')
tokenizer2 = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')
all_stopwords.append(" ")
stopwords = set(tokenizer2.tokenize(all_stopwords, is_split_into_words =True))
stopwords.update(all_stopwords)
print(stopwords)

{'ve', 'Ġany', 'should', 'Ġwhich', 'Ġd', 'on', "haven't", 'isn', 'Ġonce', 'Ġonly', 'Ġmyself', 'o', 'Ġsome', 'themselves', 'nor', 'Ġthere', 'Ġbut', 'yourself', 'y', 'most', 'such', 'those', "mightn't", 'Ġours', 'aren', 'hasn', 'Ġbetween', 'Ġown', 'Ġhim', 'Ġnow', 'why', 'Ġthrough', 'both', 'ourselves', 'had', 'mustn', 'against', 'Ġhers', 'is', "that'll", 'once', 'Ġout', 'Ġmost', 'Ġthe', 'by', 'having', 'Ġhe', 'Ġaren', 'Ġme', 'Ġwho', 'Ġboth', 'Ġneed', "you've", "aren't", 't', 'shouldn', 'Ġbefore', 'been', 'Ġagainst', 'these', 'they', 'Ġto', 'Ġall', 'Ġunder', 'Ġbecause', 'ain', 'Ġfor', 'from', 'Ġhow', 'mightn', 'shan', "don't", 'Ġthese', 'Ġwere', 'Ġdoing', 'Ġup', 'Ġnor', 'this', 'after', 'below', 'Ġa', 'have', 'Ġbe', 'above', 'n', 'Ġhaven', 'Ġt', 'Ġdoes', 'just', 'he', 'too', "didn't", 'your', 'll', 'Ġwon', 'theirs', 'between', 'you', 'Ġif', 'through', 'couldn', 'will', 'any', "'s", 'Ġwe', "wouldn't", 'Ġi', 'Ġafter', 'Ġwhom', 'Ġdown', 'Ġover', "doesn't", 'Ġthey', 'over', 'under', 'Ġtheir',

In [ ]:
highest_attrib_tokens[(highest_attrib_tokens['tokens'].str.isalpha()) & ~(highest_attrib_tokens['tokens'].isin(stopwords)) & ~(highest_attrib_tokens['tokens']==0)][:10].reset_index(drop=True)

,tokens,attribution
0,per,0.315743
1,Ġris,0.265303
2,ĠHe,0.255713
3,Ġthings,0.233263
4,Ġworld,0.176242
5,Ġautistic,0.173848
6,ogy,0.170765
7,cul,0.143677
8,Ġprevious,0.137055
9,Ġbelieves,0.116889


In [ ]:
lowest_attrib_tokens[(lowest_attrib_tokens['tokens'].str.isalpha()) & ~(lowest_attrib_tokens['tokens'].isin(stopwords)) & ~(lowest_attrib_tokens['tokens']==0)][:10].reset_index(drop=True)

,tokens,attribution
0,ĉ,-0.385827
1,ĠI,-0.297695
2,ĠNo,-0.228441
3,Ġmg,-0.153738
4,Ġsaid,-0.152039
5,Ġevents,-0.145942
6,Ġappropriate,-0.105687
7,Ġmany,-0.105013
8,YY,-0.102260
9,Ġrespect,-0.101182


Using this [notebook](https://colab.research.google.com/drive/1lktilbL1IY4nBanlzCdP8TLsBNfUsl_U?usp=sharing), we can get the files to view the aggregated attributions for the entire dataset for both the positive and negative classes. This means we summed up and averaged the attributions for every instance of any given token throughout the entire dataset (whether or not they have positive or negative attributions).

In [ ]:
df_word = pd.read_csv("/content/drive/MyDrive/cogs402longformer/results/papers/papers_attributions/longformer_emb_papers.csv")

Here we see the highest attributions for the positive class, meaning that these tokens have the most influence when the model tries to predict positive. All of these words do have relevence to A.I. related topics.

In [ ]:
df_word[:15]

,tokens,attribution
0,Ġlearning,0.163092
1,.,0.145281
2,Ġneural,0.110611
3,Ġdata,0.097347
4,",",0.077573
5,Ġthe,0.072926
6,Ġtraining,0.052609
7,Ġdataset,0.050907
8,Ġalgorithms,0.048352
9,ĠAI,0.045684


Here we see the largest attributions for the negative class, meaning that these tokens have the most influence when the model predicts negative.

In [ ]:
df_word[:-15:-1]

,tokens,attribution
30061,Ġprogramming,-0.121651
30060,Ġprogram,-0.085085
30059,Ġprograms,-0.078384
30058,Ġlanguages,-0.070023
30057,Ġlanguage,-0.054024
30056,Ġ.,-0.053213
30055,Ġcode,-0.049736
30054,Ġsoftware,-0.037241
30053,Ġcompiler,-0.030792
30052,ĠProgramming,-0.029799
